## Game Watch Project
<br>
<br>
### Starting steps:

1. Create a folder called Game-Watch and navigate to it
2. Open command prompt/terminal and create a virtual environment
3. Activete the virtual environemt
4. Install the following modules:
    - django
    - pillow
    - django-recaptcha
    - crispy-bootstrap4
5. Create a new django-project called 'GameWatch'
6. Create a the following two apps:
    - accounts
    - review
7. Create the two templates and static folders for the root and every apps (follow best practices).
8. Create base.html and 404.html tempaltes, plus their coresponding css files.
9. Create a navbar.html template and its coresponding css file, so that you can include it in the base template (root templates.includes).<br>
<hr>

### Working on the base settings:

10. Add both apps in the INSTALLED_APPS
11. Add BASE_DIR / 'templates' to the TEMPLATES.DIRS
12. Add STATICFILES_DIRS setting and pass BASE_DIR / 'static' to it<br>
<hr>

### Working on the base urls:

13. Add the following paths in GameWatch.urls.py:
    - path('accounts/', include('django.contrib.auth.urls'))
    - path('accounts/', include('accounts.urls'))
14. Create a file called urls.py in your accounts app<br>
<hr>

### Working on reCaptcha:

15. Create a new app in google reCaptcha and get the public and private keys
16. Add 'django_recaptcha' to the INSTALLED_APPS 
17. Add the following two settings:
    - RECAPTCHA_PUBLIC_KEY = "your-public-key"
    - RECAPTCHA_PRIVATE_KEY = "your-private-key"<br>
<hr>

### Adding Bootstrap to the project:

18. Add the following two to the INSTALLED_APPS:
    - 'crispy_forms'
    - 'crispy_bootstrap4'
19. Add the following two settings to the project:
    - CRISPY_ALLOWED_TEMPLATE_PACKS = "bootstrap4"
    - CRISPY_TEMPLATE_PACK = "bootstrap4"
20. Navigate to the _base.html template, populate it with standard html structure and add the following block to it:
    - page_title
    - css_files
    - content
21. At the top of _base.html template, load statics
22. Before the css_files block, add the following links to bootstrap CDN
    - <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/css/bootstrap.min.css">
    - <script src="https://cdn.jsdelivr.net/npm/jquery@3.7.1/dist/jquery.slim.min.js"></script>
    - <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.1/dist/umd/popper.min.js"></script>
    - <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/js/bootstrap.bundle.min.js"></script>
23. From now on, you should add the following to all of your templates, so you can use bootstrap filters inside DTL:
    - {% load crispy_forms_tags %}<br>
<hr>

### Creating a root template for the website:

24. Create a new template called 'index.html' and its associated css file in the review app
25. Just type some sudo-text inside the content block and give the title 'Game Watch'
26. Navigate to GameWatch.urls and add the following path afte the 'admin/' path:
    - path('', include('review.urls'), name='home')
27. Create a file called urls.py in your review app and follow the best practices for it
    - Define a path for the root url and point to view class called IndexView
28. Create a TemplateView class inside review.views.py and load index.html using it.<br>
<hr>

### Using CustomUser Model instead of django auth:

29. Add AUTH_USER_MODEL setting and pass 'accounts.CustomUser' to it.
30. Create a CustomUser model based on the AbstractUser in your accounts.models.py, the model should have the following attributes (remember you are extending the AbstractUser model, hence you don't need to define them all):
    - id (pk)
    - first_name
    - last_name
    - email
    - slug (based on email)
    - password
    - image
    - is_author
    - is_suspended (default=False)
    - Joined_on (date)
    - about (text maxlength=500)

In [ ]:
class CustomUser(AbstractUser):
    email = models.EmailField(unique=True)
    slug = models.SlugField(unique=True, db_index=True, null=False)
    image = models.ImageField(null=True, blank=True)
    is_author = models.BooleanField(default=False)
    is_suspended = models.BooleanField(default=False)
    joined_on = models.DateField(auto_now_add=True)
    about = models.TextField(max_length=500, blank=True)  

    def save(self, *args, **kwargs):
        if not self.slug:  # Only set the slug if it hasn't been set before
            self.slug = slugify(self.email)
        super().save(*args, **kwargs)

### Working on CustomUser Forms:

31. Create a new file called forms.py inside your accounts app and navigate to it
32. Import ReCaptchaField from django_recaptcha.fields
32. Craete a form-class called 'CustomUserCreationForm' and extend it from auth.forms.UserCreationForm:
    - This form should be based on the CustomUser model
33. Define the model so that on the signup-form only asked for the following fields and reCaptcha is presented as the last field:
    - email
    - first_name
    - last_name
    - password1
    - password2
    - captcha
34. All fields should be required.
35. Create a new form-class called CustomUserChangeForm and extend it from auth.forms.UserChangeForm:
    - This form should be based on the CustomUser model and uses all fields

In [ ]:
from django import forms
from .models import CustomUser
from django.contrib.auth.forms import UserCreationForm, UserChangeForm
from django_recaptcha.fields import ReCaptchaField

class CustomUserCreationForm(UserCreationForm):
    email = forms.EmailField(required=True)
    first_name = forms.CharField(max_length=50, required=True)
    last_name = forms.CharField(max_length=50, required=True)
    captcha = ReCaptchaField()

    class Meta:
        model = CustomUser
        fields = ('email', 'first_name', 'last_name', 'password1', 'password2', 'captcha', )

    def clean_email(self):
        email = self.cleaned_data.get('email')
        if CustomUser.objects.filter(email=email).exists():
            raise forms.ValidationError("This email is already in use.")
        return email

class CustomUserChangeForm(UserChangeForm):

    class Meta:
        model = CustomUser
        fields = ('email', 'first_name', 'last_name', 'image', 'is_author', 'is_suspended', 'joined_on', 'about')

Note: The clean_email function in a Django form is a custom validation method specifically for the email field. In Django forms, you can define custom validation logic for any field by creating a method named clean_<field_name>. This method allows you to add extra validation rules and raise validation errors if the input data does not meet the specified criteria.

### Wordkin on admin.py:

37. Navigate to accounts.admin.py
38. Import UserAdmin class from django.contrib.auth.admin
39. Create a class called CustomUserAdmin and extend it from UserAdmin Class:
    - This class should reference CustomUserCreationForm and CustomUserChangeForm
    - This class should be based on CustomUser model
    - For the list display use ['slug', 'first_name', 'last_name', 'email', 'is_author','is_suspended','about', 'joined_on']
    - Do not forget about fieldsets and add_fieldsets
    * slug and joined_on should be read_only
40. Register the model

In [ ]:
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin
from .forms import CustomUserCreationForm, CustomUserChangeForm
from .models import CustomUser

class CustomUserAdmin(UserAdmin):
    add_form = CustomUserCreationForm
    form = CustomUserChangeForm
    model = CustomUser
    list_display = ['slug', 'first_name', 'last_name', 'email', 'is_author', 'is_suspended', 'about', 'joined_on']
    readonly_fields = ['slug', 'joined_on']
    
    fieldsets = UserAdmin.fieldsets + (
        (None, {'fields': ('slug', 'is_author', 'is_suspended', 'about', 'joined_on')}),
    )
    add_fieldsets = UserAdmin.add_fieldsets + (
        (None, {'fields': ('email', 'first_name', 'last_name', 'password1', 'password2', 'slug', 'is_author', 'is_suspended', 'about')}),
    )

admin.site.register(CustomUser, CustomUserAdmin)

### Working on registration templates:

41. Create the following tempaltes in accounts.registration (add css files for them too):
    - Add the neccessary tags and forms!
    - Don't style anything just yet, we will do it after the first migrations!
42. Add a new path to accounts.urls.py:
    - path('signup/', views.SignUpView.as_view(), name='Signup-url')
43. Navigate to accounts.views.py and create a view-class called 'SignUpView' and extend it from 'CreateView':
    - This view class should be based on CustomUserCreationForm
    - Points to the 'registration/signup.html' template
    